In [1]:
import sys
sys.path.append('../../FourVector')
sys.path.append('../project')

from FourVector import FourVector
from ThreeVector import ThreeVector

from FutureColliderTools import SmearVertex, GetCorrectedMass, GetMissingMass2, GetQ2
from FutureColliderDataLoader import LoadData_KMuNu, LoadData_DsMuNu

from FutureColliderVariables import *

import numpy as np
import ROOT

ROOT.enableJSVis()
ROOT.gStyle.SetOptStat(0)



Welcome to JupyROOT 6.08/02


In [2]:
from prettytable import PrettyTable
def PlotHistograms(Filename):
    
    #Due to python's garbage collection the Histograms, Stacks, Files etc. need to be stored somewhere.
    global Histograms
    global Stacks
    global Files
    global legend
    global Textarr
    
    canvas.Divide(4,3)
    Histograms_norm = []
    
    table = PrettyTable()
    
    for it, resolution in enumerate(np.linspace(0.0, 1.0, 11)):
        
        # Open the histogram file for reading
        File_Toys = ROOT.TFile.Open(Filename.format(resolution), "READ")
        Files += [File_Toys]

        KeyList = [ Key for Key in File_Toys.GetListOfKeys() if "MCORR" in Key.GetName() ]
        Histograms_norm += [[ Key.ReadObj().Clone() for Key in KeyList if "_norm" in Key.GetName()]]
        Histograms +=      [[ Key.ReadObj().Clone() for Key in KeyList if "_norm" not in Key.GetName()]]

        # On the first loop, fill the Legend, and add the first column to the Table
        if it == 0:
            HistogramNames = [Hist.GetName().replace("MCORR_", "").replace("Combinatorial", "C") for Hist in Histograms[0] ]
            table.add_column("", HistogramNames)
            
            HistogramNames = [Hist.GetTitle() for Hist in Histograms_norm[0] ]
            for Name, Hist in zip(HistogramNames, Histograms_norm[0]):
                legend.AddEntry(Hist, Name, "lep")
        
        HistogramYields = [int(Hist.Integral()) for Hist in Histograms[-1] ]
        table.add_column(str(resolution), HistogramYields)
        
        #Make a stack of the Historams
        Stack = ROOT.THStack()
        Color = 1
        for Hist in Histograms_norm[it]:
            #Hist.Sumw2(False)
            Hist.SetDirectory(0)
            Hist.SetLineColor(Color)
            Hist.SetFillStyle(0)
            Color+=1
            Stack.Add(Hist)

        Stacks += [Stack]
        canvas.cd(it+1)
        Stack.Draw("nostack")
        
        Text = ROOT.TPaveText(0.1,0.7,0.5,0.9, "NDC")
        Text.AddText("#sigma (Vertex) =  "+str(resolution)+" #dot #sigma (LHCb)")
        Text.Draw()
        Textarr += [Text]
        
    canvas.cd(12)
    legend.Draw()
    print table


In [3]:
canvas = ROOT.TCanvas("c1", "c1", 1000, 600)

Histograms = []
Stacks = []
Files = []
Textarr = []
legend = ROOT.TLegend(0.1,0.1,0.9,0.9);

PlotHistograms("../output/Source_Histograms_DsMu_{0}_LHCb.root")

canvas.Draw()


+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|                    |  0.0   |  0.1   |  0.2   |  0.3   |  0.4   |  0.5   |  0.6   |  0.7   |  0.8   |  0.9   |  1.0   |
+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  13774000_Dsstar   | 169509 | 165269 | 161161 | 157309 | 153570 | 149949 | 146715 | 143327 | 140042 | 137066 | 134027 |
|    13774000_Ds     | 75167  | 72973  | 70897  | 68927  | 67013  | 65266  | 63603  | 61898  | 60328  | 58785  | 57156  |
|   13774000_DsTau   |  2317  |  2291  |  2258  |  2222  |  2199  |  2153  |  2119  |  2081  |  2077  |  2019  |  2008  |
| 13774000_DsstarTau |  4339  |  4279  |  4221  |  4157  |  4096  |  4035  |  4005  |  3936  |  3892  |  3820  |  3765  |
|  13774000_Dsstar0  | 20424  | 19988  | 19514  | 19083  | 18792  | 18350  | 17960  | 17624  | 17277  | 16896  | 16592  |
|  13774000_Dsstar1  |  

In [4]:
canvas = ROOT.TCanvas("c1", "c1", 1000, 600)

Histograms = []
Stacks = []
Files = []
legend = ROOT.TLegend(0.1,0.1,0.9,0.9);

PlotHistograms("../output/Source_Histograms_KMu_{0}_LHCb.root")

canvas.Draw()

+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|                    |  0.0   |  0.1   |  0.2   |  0.3   |  0.4   |  0.5   |  0.6   |  0.7   |  0.8   |  0.9   |  1.0   |
+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  13774000_Dsstar   | 292436 | 289105 | 285792 | 282680 | 279378 | 276447 | 273589 | 270853 | 267957 | 264957 | 262255 |
|      15574010      |  2166  |  2147  |  2120  |  2107  |  2098  |  2056  |  2036  |  2020  |  2000  |  1992  |  1950  |
|      12513010      | 85537  | 84570  | 83442  | 82531  | 81547  | 80622  | 79602  | 78799  | 77984  | 76999  | 76177  |
|    13774000_Ds     | 118863 | 117575 | 116304 | 115107 | 113877 | 112612 | 111575 | 110322 | 109212 | 108173 | 107039 |
|      13512010      | 628487 | 612028 | 596422 | 581348 | 567284 | 553661 | 541351 | 528810 | 517365 | 506280 | 495640 |
|   13774000_DsTau   |  

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# Combining/Removing Histograms

### We can see that several of the templates have similar shapes.

### The following can be combined or removed:

## For $B_s \to D_s^-  \mu^+ \nu_{\mu} $

* Both $B_s \to D_s \tau X$ modes
* Both Further Excited Ds* resonance

## For $B_s \to K^+  \mu^+ \nu_{\mu} $
* Both $B_s \to D_s \tau X$ modes
* Both Combinatorial Modes
* Both Further Excited Ds* resonances and the Lambda Mode
* $B_s \to D_s  \mu^+ \nu_{\mu} $ and $B_s \to D_s^*  \mu^+ \nu_{\mu} $



In [5]:
canvas = ROOT.TCanvas("c1", "c1", 1000, 600)

Histograms = []
Stacks = []
Files = []
legend = ROOT.TLegend(0.1,0.1,0.9,0.9);

PlotHistograms("../output/Source_Histograms_DsMu_{0}_LHCb_Merged.root")

canvas.Draw()

+-----------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|                 |  0.0   |  0.1   |  0.2   |  0.3   |  0.4   |  0.5   |  0.6   |  0.7   |  0.8   |  0.9   |  1.0   |
+-----------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|        C        |  738   |  736   |  737   |  728   |  738   |  740   |  739   |  736   |  724   |  734   |  718   |
|   13774000_Tau  |  6656  |  6570  |  6479  |  6379  |  6295  |  6188  |  6124  |  6017  |  5969  |  5839  |  5773  |
| 13774000_Dsstar | 169509 | 165269 | 161161 | 157309 | 153570 | 149949 | 146715 | 143327 | 140042 | 137066 | 134027 |
|  13774000_Ds01  | 26745  | 26210  | 25613  | 25074  | 24675  | 24101  | 23621  | 23153  | 22748  | 22264  | 21837  |
|   13774000_Ds   | 75167  | 72973  | 70897  | 68927  | 67013  | 65266  | 63603  | 61898  | 60328  | 58785  | 57156  |
+-----------------+--------+--------+--------+--

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [6]:
canvas = ROOT.TCanvas("c1", "c1", 1000, 600)

Histograms = []
Stacks = []
Files = []
legend = ROOT.TLegend(0.1,0.1,0.9,0.9);

PlotHistograms("../output/Source_Histograms_KMu_{0}_LHCb_Merged.root")

canvas.Draw()

+---------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|               |  0.0   |  0.1   |  0.2   |  0.3   |  0.4   |  0.5   |  0.6   |  0.7   |  0.8   |  0.9   |  1.0   |
+---------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| 13774000_Ds01 | 44005  | 43608  | 43266  | 42905  | 42585  | 42095  | 41784  | 41382  | 41200  | 40889  | 40452  |
|       C       | 59681  | 59689  | 59710  | 59696  | 59729  | 59695  | 59670  | 59704  | 59652  | 59586  | 59595  |
|    12513010   | 85537  | 84570  | 83442  | 82531  | 81547  | 80622  | 79602  | 78799  | 77984  | 76999  | 76177  |
|  13774000_Tau |  6761  |  6790  |  6793  |  6806  |  6855  |  6870  |  6883  |  6831  |  6837  |  6877  |  6885  |
|    13512010   | 628487 | 612028 | 596422 | 581348 | 567284 | 553661 | 541351 | 528810 | 517365 | 506280 | 495640 |
|  13774000_DsX | 411299 | 406680 | 402096 | 397787 | 393255 | 3

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
